This file wrangles the new metadata Rachel sent me and adds it to the metadata I already have.

In [1]:
import pandas as pd
import numpy as np

In [2]:
outfile = '../../data/clean/rosen.clinical_metadata.clean'

In [3]:
ptfile = '../../final/supp_files/patients_with_sites_sampled.csv'
pt_outmeta = '../../final/supp_files/patient_clinical_metadata.csv'

# K23 data

I got three versions of this file with similar metadata, and one version with the GI symptoms scores. Need to consolidate all patients in the first three files, and merge with the GI symptoms data.

In [4]:
# This was the respiratory study, so these patients have more extensive
# respiratory symptoms and they also will have BAL samples
fk23_v1 = '../../data/raw/metadata/reviewer-response-jan2019/k23 data.csv'
fk23_v2 = '../../data/raw/metadata/reviewer-response-jan2019/FINAL AIM 2, 3, 4 BASELINE DATA.csv'
fk23_v3 = '../../data/raw/metadata/reviewer-response-jan2019/UPDATED FINAL AIM 2, 3, 4 BASELINE DATA.csv'
fk23_v4 = '../../data/raw/metadata/reviewer-response-jan2019/K23 SYMPTOM QUESTIONNAIRES.csv'

dfk23_v1 = pd.read_csv(fk23_v1)
dfk23_v2 = pd.read_csv(fk23_v2)
dfk23_v3 = pd.read_csv(fk23_v3)
# This one is tab-delimited
dfk23_v4 = pd.read_csv(fk23_v4, sep='\t')

# Add metadata
c = 'metadata_file'
c2 = 'metadata_id'

# K23 dataframes
dfk23_v1[c] = fk23_v1.rsplit('/', 1)[1]
dfk23_v1[c2] = 'k23_v1'

dfk23_v2[c] = fk23_v2.rsplit('/', 1)[1]
dfk23_v2[c2] = 'k23_v2'

dfk23_v3[c] = fk23_v3.rsplit('/', 1)[1]
dfk23_v3[c2] = 'k23_v3'

dfk23_v4[c] = fk23_v4.rsplit('/', 1)[1]
dfk23_v4[c2] = 'k23_v4'

In [5]:
k23_dfs = [dfk23_v1, dfk23_v2, dfk23_v3, dfk23_v4]
print([i.shape for i in k23_dfs])

[(154, 238), (156, 238), (156, 238), (156, 58)]


Okay, k23_v4 has different columns than the other, so I'll need to combine that with the other k23 dataframes.

In [6]:
# Make one big K23 dataframe
k23_metas = ['k23_v1', 'k23_v2', 'k23_v3']

k23_bigdf = pd.concat([dfk23_v1, dfk23_v2, dfk23_v3], sort=False)
print(k23_bigdf.shape, k23_bigdf['SUBJID'].unique().shape)

k23_bigdf = k23_bigdf.reset_index(drop=True)
remove_ix = []
for subj, subj_df in k23_bigdf.groupby('SUBJID'):

    if subj_df.shape[0] > 1:
        
        meta_ids = subj_df['metadata_id'].sort_values().values.tolist()
        keep_meta = meta_ids[-1]
        rmv_ix = subj_df.query('metadata_id != @keep_meta').index.tolist()
        remove_ix += rmv_ix


((466, 238), (156,))


In [7]:
print(len(remove_ix), k23_bigdf.shape)
k23_bigdf = k23_bigdf.drop(index=remove_ix)
print(k23_bigdf.shape, k23_bigdf['SUBJID'].unique().shape)

(310, (466, 238))
((156, 238), (156,))


In [8]:
# Now, merge with the symptoms data
print(k23_bigdf.shape, dfk23_v4.shape)
print([i for i in k23_bigdf['SUBJID'].unique() if i not in dfk23_v4['SUBJID'].values])
print([i for i in dfk23_v4['SUBJID'].values if i not in k23_bigdf['SUBJID'].unique()])

((156, 238), (156, 58))
['04-299-7']
['04-299-1']


In [9]:
common_cols = [i for i in k23_bigdf.columns if i in dfk23_v4.columns]
print(len(common_cols))
common_cols

10


['SUBJID',
 'SUBJINTL',
 'AIM1YN',
 'AIM2YN',
 'AIM3YN',
 'VISITDT',
 'VISITNO',
 'INITIAL',
 'metadata_file',
 'metadata_id']

In [10]:
# Merge outer
k23_bigdf_withsymp = pd.merge(
    k23_bigdf, dfk23_v4, 
    left_on='SUBJID', right_on='SUBJID',
    suffixes=('_allk23', '_k23symptoms'),
    how='outer')

In [11]:
k23_bigdf_withsymp.shape

(157, 295)

In [12]:
print(k23_bigdf_withsymp.shape)
k23_bigdf_withsymp.head()

(157, 295)


,SUBJID,CLAIRESEQUENC,SUBJINTL_allk23,AIM1YN_allk23,AIM2YN_allk23,AIM3YN_allk23,VISITDT_allk23,VISITNO_allk23,COMPDATE,INITIAL_allk23,...,GIPed10,GIPedSympTot,GIPed11,GIPed12,GIPed13,GIPed14,GIPedTotal,VAR00001,metadata_file_k23symptoms,metadata_id_k23symptoms
0,04-006-9,1.0,PL,0,1,0,9/21/2009,1,9/21/2009,,...,,,,,,,,,K23 SYMPTOM QUESTIONNAIRES.csv,k23_v4
1,04-009-2,1.0,CP,0,1,0,10/19/2009,1,10/19/2009,,...,,,,,,,,,K23 SYMPTOM QUESTIONNAIRES.csv,k23_v4
2,04-011-3,1.0,KG,0,1,0,11/18/2009,1,11/18/2009,,...,,,,,,,,,K23 SYMPTOM QUESTIONNAIRES.csv,k23_v4
3,04-013-4,1.0,,,,,3/4/2010,,3/4/2010,,...,,,,,,,,,K23 SYMPTOM QUESTIONNAIRES.csv,k23_v4
4,04-020-3,1.0,JPN,0,1,0,4/21/2010,1,4/21/2010,KMH,...,,,,,,,,,K23 SYMPTOM QUESTIONNAIRES.csv,k23_v4


In [13]:
# Update the metadata ID
k23_bigdf_withsymp.groupby(['metadata_id_allk23', 'metadata_id_k23symptoms']).size()

metadata_id_allk23  metadata_id_k23symptoms
k23_v3              k23_v4                     155
dtype: int64

In [14]:
k23_bigdf_withsymp['metadata_id'] = 'k23'

In [15]:
c1 = 'VISITDT_allk23'
c2 = 'VISITDT_k23symptoms'

# look at rows which have mismatch visit dates
k23_bigdf_withsymp[
   k23_bigdf_withsymp[c1].replace(' ', np.nan).fillna(k23_bigdf_withsymp[c2]) 
    != k23_bigdf_withsymp[c2].replace(' ', np.nan).fillna(k23_bigdf_withsymp[c1])
][[c1, c2, 'DOB']]

#k23_bigdf_withsymp[k23_bigdf_withsymp[c1].fillna(k23_bigdf_withsymp[c2]) != k23_bigdf_withsymp[c2].fillna(k23_bigdf_withsymp[c1])]
#k23_bigdf_withsymp.query("SUBJID == '04-013-4'")[[c1, c2]]



,VISITDT_allk23,VISITDT_k23symptoms,DOB
32,9/17/2011,9/14/2011,1/7/2010
81,8/8/2013,8/8/82013,6/15/2009
116,3/14/2011,3/14/2001,11/25/2004


Okay, there are only 3 patients with conflicting visit dates. Given that the visit date from the symptoms metadata has the one incorrect date (it has visit date in 2001 for a patient born in 2004), I feel pretty confident saying that the other source of data (i.e. the combined k23 v1, v2, and v3) is the correct visit date.

In [16]:
k23_bigdf_withsymp['visit_date'] = k23_bigdf_withsymp['VISITDT_allk23'].fillna(k23_bigdf_withsymp['VISITDT_k23symptoms'])
k23_bigdf_withsymp.shape, k23_bigdf_withsymp.dropna(subset=['visit_date']).shape

((157, 297), (157, 297))

# Other data sources

In [17]:
# R01 study is different, and focuses more on general infections. So it has
# more metadata on infections but less on respiratory symptoms. Also, these
# patients probably won't have BAL.
r01base_v1 = '../../data/raw/metadata/reviewer-response-jan2019/baselineR01.csv'
# This one is actually tab delimited
r01base_v2 = '../../data/raw/metadata/reviewer-response-jan2019/baselineforclaire-r01-pgsq.csv'

r01aim3_v1 = '../../data/raw/metadata/reviewer-response-jan2019/baselineforclaire-r01-aim3supp.csv'
# This one is actually tab delimited
r01aim3_v2 = '../../data/raw/metadata/reviewer-response-jan2019/baselineforclaire-r01-aim3supp-pgsq.csv'

r01long = '../../data/raw/metadata/reviewer-response-jan2019/r01 longitudinal.csv'

# Fundoplication patients
fundo_v1 = '../../data/raw/metadata/reviewer-response-jan2019/fundobaselinedata_1.csv'
fundo_v2 = '../../data/raw/metadata/reviewer-response-jan2019/updatedfundobaselinedata.csv'

dfr01base_v1 = pd.read_csv(r01base_v1)
dfr01base_v2 = pd.read_csv(r01base_v2, sep='\t')

dfr01long = pd.read_csv(r01long)

dfr01aim3_v1 = pd.read_csv(r01aim3_v1)
dfr01aim3_v2 = pd.read_csv(r01aim3_v2, sep='\t')

dffundo_v1 = pd.read_csv(fundo_v1)
dffundo_v2 = pd.read_csv(fundo_v2)

# Fix a few things in the fundo dataframe
dffundo_v1 = dffundo_v1.rename(columns={'SUBJID_BL':"SUBJID"})
dffundo_v2 = dffundo_v2.rename(columns={'SUBJID_BL':"SUBJID"})

dffundo_v1['SUBJID'] = dffundo_v1['SUBJID'].str.replace(' ', '-')
dffundo_v2['SUBJID'] = dffundo_v2['SUBJID'].str.replace(' ', '-')

In [18]:
# Label metadata source in each df
c = 'metadata_file'
c2 = 'metadata_id'

# R01 dataframes
dfr01base_v1[c] = r01base_v1.rsplit('/', 1)[1]
dfr01base_v1[c2] = 'r01_baseline_v1'

dfr01base_v2[c] = r01base_v2.rsplit('/', 1)[1]
dfr01base_v2[c2] = 'r01_baseline_v2'

dfr01long[c] = r01long.rsplit('/', 1)[1]
dfr01long[c2] = 'r01_longitudinal'

dfr01aim3_v1[c] = r01aim3_v1.rsplit('/', 1)[1]
dfr01aim3_v1[c2] = 'r01_aim3_v1'

dfr01aim3_v2[c] = r01aim3_v2.rsplit('/', 1)[1]
dfr01aim3_v2[c2] = 'r01_aim3_v2'

# Fundo
dffundo_v1[c] = fundo_v1.rsplit('/', 1)[1]
dffundo_v1[c2] = 'fundo_v1'

dffundo_v2[c] = fundo_v2.rsplit('/', 1)[1]
dffundo_v2[c2] = 'fundo_v2'

In [19]:
alldfs = [k23_bigdf_withsymp,
          dfr01aim3_v1, dfr01aim3_v2, 
          dfr01base_v1, dfr01base_v2,
          dfr01long, 
          dffundo_v1, dffundo_v2]

print([i.shape for i in alldfs])

bigdf = pd.concat(alldfs, sort=False)
print(bigdf.shape, bigdf['SUBJID'].dropna().shape, bigdf['SUBJID'].unique().shape)

[(157, 297), (23, 371), (22, 371), (16, 205), (16, 206), (22, 371), (26, 531), (26, 531)]
((308, 1221), (308,), (223,))


In [20]:
#for d in alldfs:
#    display(d.head())

## Remove duplicate patients

In [21]:
# See what combination of metadata files each patient has

# make sure bigdf index doesn't have duplicates
bigdf = bigdf.reset_index(drop=True)

for subj, subj_df in bigdf.groupby('SUBJID'):
    meta_ids = subj_df['metadata_id'].sort_values().values
    meta_ids = '-'.join(meta_ids)
    bigdf.loc[subj_df.index, 'subject_meta_files'] = meta_ids

#subj_df = bigdf.query('SUBJID == "01-058-2"')
#meta_ids = subj_df['metadata_id'].sort_values().values
#meta_ids
bigdf.groupby('subject_meta_files').size()

subject_meta_files
fundo_v1-fundo_v2                                 52
k23                                              157
r01_aim3_v1-r01_aim3_v2                            6
r01_aim3_v1-r01_aim3_v2-r01_longitudinal          57
r01_aim3_v1-r01_longitudinal-r01_longitudinal      3
r01_baseline_v1-r01_baseline_v2                   32
r01_longitudinal                                   1
dtype: int64

Okay, so all fundoplication patients are in both files: use only fundo_v2.

All R01 baseline patients are in both versions: use only r01_baseline_v2

K23 patients is taken care of already.

Other R01 patients are also more complicated:
- if they have r01aim3_v2, use that (it's the most recent file I got from the team)
- the one patient that is present twice in r01_longitudinal and also in r01_aim3_v1 is 03-200-1, which is not actually used in any of my analyses. So we'll just remove this one.

In [22]:
# Go through and grab all the idx's we want to remove

# # make sure bigdf index doesn't have duplicates
# bigdf = bigdf.reset_index(drop=True)
remove_ix = []
for subj, subj_df in bigdf.groupby('SUBJID'):
    if subj_df.shape[0] > 1:
        
        meta_ids = subj_df['metadata_id'].sort_values().values.tolist()
        
        # If fundo patients, keep fundo_v2
        if 'fundo_v1' in meta_ids or 'fundo_v2' in meta_ids:
            keep_meta = meta_ids[-1]
            rmv_ix = subj_df.query('metadata_id != @keep_meta').index.tolist()
            remove_ix += rmv_ix
        
        # If this is R01 baseline, use r01_baseline_v2
        elif 'r01_baseline_v1' in meta_ids or 'r01_baseline_v2' in meta_ids:
            keep_meta = meta_ids[-1]
            rmv_ix = subj_df.query('metadata_id != @keep_meta').index.tolist()
            remove_ix += rmv_ix
        
        # If they have r01aim3_v2, keep that
        elif 'r01_aim3_v2' in meta_ids:
            keep_meta = 'r01_aim3_v2'
            rmv_ix = subj_df.query('metadata_id != @keep_meta').index.tolist()
            remove_ix += rmv_ix
        
        else:
            print(subj, meta_ids)

('03-200-1', ['r01_aim3_v1', 'r01_longitudinal', 'r01_longitudinal'])


In [23]:
print(len(remove_ix), bigdf.shape)
bigdf_nodup = bigdf.drop(index=remove_ix)
print(bigdf_nodup.shape, bigdf_nodup['SUBJID'].unique().shape)

(83, (308, 1222))
((225, 1222), (223,))


In [24]:
tmp = bigdf_nodup.groupby('SUBJID').size()
tmp[tmp > 1]

SUBJID
03-200-1    3
dtype: int64

# Correct subject IDs with discrepancies

Here, the left subject ID is the ones I have in my data and the explanation is where I should find them in this data.

```
01-297-4 - "01-297-4 is actually 01-297-2 - this is in the "r01" database"
029-6-F1 - "please note that 029-6 is listed in database as 029-5"
03-125-1 - "03-125-9 (we believe this was listed as 03-125-1 but the correct ID should be 03-125-9 based on our sample logs)"
04-164-1 - "04-164-1 is listed in the database as 02-164-1"
04-168-4 - "04-168-4 is listed in the database as 14-168-4"
04-235-8 - "04-235-8 is listed in the database as 04-235-4"
13-058-2 - "13-058-2 - listed in the "r01" database as 01-058-2"
13-089-1 - "13-089-1 - listed in the "r01" database as 01-089-1"
14-233-0 - "14-233-0 is listed in the database as 04-233-1"
```

Note that because I have both 04-164-1 and 02-164-1 in my data, I'll need to add an extra row for this "new subject" in this data (I think - we'll see, this is mostly a note for me).

In [25]:
bigdf_nodup['subject_id'] = bigdf_nodup['SUBJID']

In [26]:
repdict = {
    '01-297-2': '01-297-4',
    '029-5-F1': '029-6-F1',
    '03-125-9': '03-125-1',
    #'02-164-1': '04-164-1',
    '14-168-4': '04-168-4',
    '04-235-4': '04-235-8',
    '01-058-2': '13-058-2',
    '01-089-1': '13-089-1',
    '04-233-1': '14-233-0'
}

In [27]:
[k for k in repdict if k not in bigdf_nodup['SUBJID'].values]

[]

In [28]:
bigdf_nodup['subject_id'] = bigdf_nodup['subject_id'].replace(repdict)

In [29]:
# Add a new row for 04-164-1, that's just the same as 02-164-1
newrow = bigdf_nodup.query('SUBJID == "02-164-1"')
newrow['subject_id'] = '04-164-1'
bigdf_nodup = bigdf_nodup.append(newrow, ignore_index=True)

/Users/claire/anaconda/envs/aspiration/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
# And add a row full of NaN's for 'dup05-013-1'
newrow = pd.Series(index=bigdf_nodup.columns)
newrow['subject_id'] = 'dup05-013-1'
newrow['SUBJID'] = 'dup05-013-1'
bigdf_nodup = bigdf_nodup.append(newrow, ignore_index=True)

In [31]:
bigdf_nodup.shape

(227, 1223)

In [32]:
# Keep only the 222 patients I used in my analyses
fpatients = '../../final/supp_files/combined_patient_site_and_analyses.csv'
dfpts = pd.read_csv(fpatients, index_col=0)
#dfpts.head()

In [33]:
dfpts.shape

(220, 32)

In [34]:
analysis_subjs = dfpts.index.tolist()
bigdf_nodup = bigdf_nodup.query('subject_id == @analysis_subjs')
bigdf_nodup.shape

(220, 1223)

# Reconcile different gender codings

K23 data:    
GENDER    
- 1 = M
- 2 = F
- -6 = missing

R01 data:    
GENDER
- 0 = M
- 1 = F


In [35]:
bigdf_nodup.query('metadata_id == "k23"').groupby(['metadata_id', 'GENDER']).size()

metadata_id  GENDER
k23                     1
             -6         1
             1         92
             2         62
dtype: int64

In [36]:
# Replace the K23 codes in-column
k23_df = bigdf_nodup.query('metadata_id == "k23"')

bigdf_nodup.loc[k23_df.index, 'GENDER'] = k23_df['GENDER'].replace({'1': 0, '2': 1, '-6': np.nan, ' ': np.nan})

# Consolidate metadata

Now that I have all the right rows, I'll need to consolidate the metadata labels from the different files.

For each metadata category, I'll need to:

- gather all the relevant columns
- check that there are no duplicate entries for each patient (i.e. they don't have data in two of the columns)
    - consolidate duplicates when there are any
- make a combine metadata column

Also, from Eliza's latest email on Feb 7, 2019: "We spoke with Rachel and have discussed that any -8 and -6 should be no’s and -9 or missing data should be considered missing/unknown."

In [37]:
def identify_conflicts(row, metacols):
    """
    Return whether or not there is conflicting metadata
    in the metacols of the given row.

    Return True if there is a discrepancy and False if
    they all match. Return nan if they are all nan.
    """

    if row[metacols].isnull().sum() == len(metacols):
        return np.nan

    tmp = row[metacols].dropna()#.apply(lambda x: ppimap[x])

    # tmp is a Series, so this is basically only if you only
    # gave one column
    if tmp.shape[0] == 1:
        return False

    elif tmp.eq(tmp.iloc[0]).sum() == tmp.shape[0]:
        # If all values in the row are equal, no conflict
        return False
    else:
        return True

In [38]:
newcols = []

## Demographics

## Calculate Age

From Eliza Fishman's email on 2/1/19, the following columns indicate date of sampling:

- FUNDO: COMPDATE_BL
- R01: BASELINEDATE
- K23: VISITDT

I already consolidated the VISITDT's from the different k23 files into a new column called "visit_date"


In [39]:
bigdf_nodup['dob_all'] = bigdf_nodup['DOB'].fillna(bigdf_nodup['DOB_BL'])
bigdf_nodup['dob_all'].dropna().shape, bigdf_nodup.shape

((220,), (220, 1224))

In [40]:
bigdf_nodup['metadata_id'].unique()

array(['k23', 'r01_aim3_v2', 'r01_baseline_v2', 'r01_longitudinal',
       'fundo_v2'], dtype=object)

In [41]:
meta_id = 'fundo_v2'
date_id = 'COMPDATE_BL'

tmp_age = bigdf_nodup.query('metadata_id == @meta_id')
tmp_age = (
    pd.to_datetime(
        tmp_age[date_id].replace(' ', np.nan)
    ) 
    - pd.to_datetime(
        tmp_age['dob_all'].replace(' ', np.nan)
    )).dt.days / 365.0
bigdf_nodup.loc[tmp_age.index, 'age'] = tmp_age.values

meta_id = ['k23']
date_id = 'visit_date'

tmp_age = bigdf_nodup.query('metadata_id == @meta_id')
tmp_age = (
    pd.to_datetime(
        tmp_age[date_id].replace(' ', np.nan)
    ) 
    - pd.to_datetime(
        tmp_age['dob_all'].replace(' ', np.nan)
    )).dt.days / 365.0
bigdf_nodup.loc[tmp_age.index, 'age'] = tmp_age.values

meta_id = ['r01_aim3_v2', 'r01_baseline_v2', 'r01_longitudinal']
date_id = 'BASELINEDATE'

tmp_age = bigdf_nodup.query('metadata_id == @meta_id')
tmp_age = (
    pd.to_datetime(
        tmp_age[date_id].replace(' ', np.nan)
    ) 
    - pd.to_datetime(
        tmp_age['dob_all'].replace(' ', np.nan)
    )).dt.days / 365.0
bigdf_nodup.loc[tmp_age.index, 'age'] = tmp_age.values

Check discrepancies

In [42]:
#bigdf_nodup[['subject_id', 'age', 'metadata_id']].sort_values('age').head()

In [43]:
#bigdf_nodup[['subject_id', 'age', 'metadata_id']].sort_values('age').tail(15)

In [44]:
date_cols = ['subject_id', 'age', 'COMPDATE_BL', 'visit_date', 'BASELINEDATE', 'dob_all', 'metadata_id']

missing_subjs = (
    bigdf_nodup.query('age < 0')['subject_id'].values.tolist() +
    bigdf_nodup.query('age > 18')['subject_id'].values.tolist() + 
    bigdf_nodup[['age', 'subject_id']].fillna('nan').query('age == "nan"')['subject_id'].values.tolist()
)

bigdf_nodup.query('subject_id == @missing_subjs')[date_cols].rename(columns={
    'age': 'calculated_age',
    'visit_date': 'VISITDT_k23',
    'dob_all': 'DOB'
}).sort_values(by='calculated_age')

,subject_id,calculated_age,COMPDATE_BL,VISITDT_k23,BASELINEDATE,DOB,metadata_id
223,057-9-F1,-0.419178,5/22/2015,NaN,NaN,10/22/2015,fundo_v2
76,04-168-4,18.117808,NaN,10/31/2012,NaN,9/23/1994,k23
172,03-153-7,18.120548,NaN,NaN,10/24/2014,9/14/1996,r01_aim3_v2
208,032-1-F1,18.602740,7/3/2013,NaN,NaN,11/30/1994,fundo_v2
176,03-182-8,19.364384,NaN,NaN,2/5/2015,9/30/1995,r01_aim3_v2
209,033-8-F1,19.465753,7/22/2013,NaN,NaN,2/7/1994,fundo_v2
214,041-3-F1,23.252055,10/17/2013,NaN,NaN,7/23/1990,fundo_v2
219,048-1-F1,23.361644,5/27/2014,NaN,NaN,1/21/1991,fundo_v2
87,04-234-5,23.490411,NaN,5/12/2014,NaN,11/20/1990,k23
50,04-129-7,NaN,NaN,4/19/2012,NaN,,k23


From Eliza's email on 2/11,

> Patient missing DOB = 04-129-7 : 10/23/2006    
> Patients missing visit date = 021-7-F1 = 7/19/2012 and 026-8-F1=8/23/2012      
> Patient with negative age = 057-9-F1 birthday is 10/22/2014 and their date of visit is 5/22/2015 - there should not be a negative age, must have been an error in the birth year.     
> All the others on this table are older than 18 have the correct ages.


In [45]:
bigdf_nodup.query('SUBJID == "04-129-7"')[['DOB'] + date_cols]

,DOB,subject_id,age,COMPDATE_BL,visit_date,BASELINEDATE,dob_all,metadata_id
50,,04-129-7,NaN,NaN,4/19/2012,NaN,,k23


In [46]:
idx = bigdf_nodup.query('SUBJID == "04-129-7"').index
bigdf_nodup.loc[idx, 'dob_all'] = "10/23/2006"
bigdf_nodup.loc[idx, 'age'] = (
    (pd.to_datetime(bigdf_nodup.loc[idx, 'visit_date']) 
     - pd.to_datetime(bigdf_nodup.loc[idx, 'dob_all'])).dt.days / 365.0
)
bigdf_nodup.loc[idx]

,SUBJID,CLAIRESEQUENC,SUBJINTL_allk23,AIM1YN_allk23,AIM2YN_allk23,AIM3YN_allk23,VISITDT_allk23,VISITNO_allk23,COMPDATE,INITIAL_allk23,...,ABX1SP,ABX1DUR,ABX1DOSE,ABX2SP,ABX2DUR,ABX2DOSE,subject_meta_files,subject_id,dob_all,age
50,04-129-7,1.0,MB,0,1,0,4/19/2012,1,4/19/2012,,...,NaN,NaN,NaN,NaN,NaN,NaN,k23,04-129-7,10/23/2006,5.493151


In [47]:
idx = bigdf_nodup.query('SUBJID == "021-7-F1"').index
bigdf_nodup.loc[idx, 'COMPDATE_BL'] = "7/19/2012"
bigdf_nodup.loc[idx, 'age'] = (
    (pd.to_datetime(bigdf_nodup.loc[idx, 'COMPDATE_BL']) 
     - pd.to_datetime(bigdf_nodup.loc[idx, 'dob_all'])).dt.days / 365.0
)
bigdf_nodup.loc[idx, date_cols]

,subject_id,age,COMPDATE_BL,visit_date,BASELINEDATE,dob_all,metadata_id
199,021-7-F1,6.991781,7/19/2012,NaN,NaN,7/24/2005,fundo_v2


In [48]:
idx = bigdf_nodup.query('SUBJID == "026-8-F1"').index
bigdf_nodup.loc[idx, 'COMPDATE_BL'] = "8/23/2012"
bigdf_nodup.loc[idx, 'age'] = (
    (pd.to_datetime(bigdf_nodup.loc[idx, 'COMPDATE_BL']) 
     - pd.to_datetime(bigdf_nodup.loc[idx, 'dob_all'])).dt.days / 365.0
)
bigdf_nodup.loc[idx, date_cols]

,subject_id,age,COMPDATE_BL,visit_date,BASELINEDATE,dob_all,metadata_id
203,026-8-F1,2.830137,8/23/2012,NaN,NaN,10/25/2009,fundo_v2


In [49]:
idx = bigdf_nodup.query('SUBJID == "057-9-F1"').index
bigdf_nodup.loc[idx, 'dob_all'] = "10/22/2014"
bigdf_nodup.loc[idx, 'COMPDATE_BL'] = "5/22/2015"
bigdf_nodup.loc[idx, 'age'] = (
    (pd.to_datetime(bigdf_nodup.loc[idx, 'COMPDATE_BL']) 
     - pd.to_datetime(bigdf_nodup.loc[idx, 'dob_all'])).dt.days / 365.0
)
bigdf_nodup.loc[idx, date_cols]


,subject_id,age,COMPDATE_BL,visit_date,BASELINEDATE,dob_all,metadata_id
223,057-9-F1,0.580822,5/22/2015,NaN,NaN,10/22/2014,fundo_v2


In [50]:
newcols.append('age')

### Margot's question on 2/19

```
I have a question about the table that you sent regarding which patients' samples have sequencing data. Some of the study IDs have multiple dates sent for sequencing - are you able to see which dates got microbiome data?

The IDs with multiple dates are:
01-112-7 (aka 13-112-7)
01-164-7 (aka 13-164-7)
01-165-8 (aka 13-165-8)
01-200-1 (may be listed as 03-200-1 or 13-200-1)
01-230-9
01-263-4
13-058-2
13-089-1
13-199-7 (aka 01-199-7 or 03-199-7)
```

In [54]:
c = ['subject_id', "BASELINEDATE"]
s = ["01-112-7",
     "13-112-7",
     "01-164-7",
     "01-165-8",
     "13-164-7",
     "01-200-1", 
     "03-200-1",
     "13-200-1",
     "01-230-9",
     "01-263-4",
     "13-058-2",
     "13-089-1",
     "13-199-7",
     "01-199-7",
     "03-199-1"
    ]
bigdf_nodup.query('subject_id == @s')[c]

,subject_id,BASELINEDATE
180,13-058-2,10/14/2013
181,13-089-1,1/16/2014
182,01-112-7,4/14/2014
183,01-164-7,11/5/2014
184,01-165-8,11/6/2014
187,01-200-1,4/1/2015
190,01-230-9,7/21/2015
192,01-263-4,9/16/2015


### Gender

The following patients don't have gender data in the table, but from Eliza's email on Feb 1 they are all male: 04-062-8, 04-103-5, 026-8-F1

In [140]:
# First, replace GENDER column so that 0 = M and 1 = F
# (to match the )

cols = ['GENDER', 'GENDER_BL']
newcol = 'gender_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]])
else:
    print('conflicts!')
    
# Replace with string
bigdf_nodup[newcol] = bigdf_nodup[newcol].replace({'0': 'M', '1': 'F', 0.0: 'M', 1.0: 'F'})

# Manually fill in patients with missing data    
male_pts = ['04-062-8', '04-103-5', '026-8-F1']
bigdf_nodup.loc[bigdf_nodup.query('subject_id == @male_pts').index, 'gender_all'] = 'M'


In [141]:
bigdf_nodup.groupby(newcol).size()

gender_all
2      1
F     90
M    129
dtype: int64

In [142]:
bigdf_nodup.query('gender_all == "2"')

,SUBJID,CLAIRESEQUENC,SUBJINTL_allk23,AIM1YN_allk23,AIM2YN_allk23,AIM3YN_allk23,VISITDT_allk23,VISITNO_allk23,COMPDATE,INITIAL_allk23,...,ABX1DUR,ABX1DOSE,ABX2SP,ABX2DUR,ABX2DOSE,subject_meta_files,subject_id,dob_all,age,gender_all
206,029-5-F1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,fundo_v1-fundo_v2,029-6-F1,2/17/2012,1.131507,2


From Eliza's email on 2/11,

> Gender: 029-5-F1= Female

In [143]:
bigdf_nodup.loc[bigdf_nodup.query('SUBJID == "029-5-F1"').index, 'gender_all'] = "F"

In [144]:
bigdf_nodup.groupby(newcol).size()

gender_all
F     91
M    129
dtype: int64

## Medications

In [145]:
# Recent history of antibiotics
# -6: and -8 are no's; anything else is unkown
cols = ['ANTIB1MO', 'T0ABX1MO', 'ANTIB1MONTH']
newcol = 'abx_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]]).fillna(bigdf_nodup[cols[2]])
else:
    print('conflicts!')
    
# On PPIs now
cols = ['MEDI2C', 'PPI48HRS', 'PPIstatus_BL']
newcol = 'ppi_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]]).fillna(bigdf_nodup[cols[2]])
else:
    print('conflicts!')

# H2 blockers
# -6: 0, -8: unknown
cols = ['MEDI1C', 'H22MOS', 'MEDI1C_BL']
newcol = 'h2blockers_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]]).fillna(bigdf_nodup[cols[2]])
else:
    print('conflicts!')
    
# Recent usage of inhaled steroids
cols = ['MEDI3C', 'INHAL2MOS', 'MEDI3C_BL']
newcol = 'inhaled_steroids_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]]).fillna(bigdf_nodup[cols[2]])
else:
    print('conflicts!')

# Recent usage of oral steroids

# First, I need to consolidate the R01 data, where
# this is broken up into two columns (SWALL2MOS and ORAL2MOS)
def consolidate_oral_steroids(row):
    '''
    Consolidate the two oral steroids rows from R01 data
    
    If either column has a "1", return that. 
    If not, but at least one column has a "0", return that.
        (Because the patient's parents knew about at least one
        of the two types of delivery methods, so we'll upweight 
        that response over the other "I don't know". Also, most
        of the ORAL2MOS responses are -9 [don't know].)

    If neither column has a 1 or 0, return NaN.    
    '''
    vals = row[['SWALL2MOS', 'ORAL2MOS']].values
    if '1' in vals:
        return '1'
    elif '0' in vals:
        return '0'
    else:
        return np.nan
    
bigdf_nodup['swall_or_oral_2mos'] = bigdf_nodup[['SWALL2MOS', 'ORAL2MOS']].apply(lambda row: consolidate_oral_steroids(row), axis=1)


cols = ['MEDI4C', 'swall_or_oral_2mos', 'MEDI4C_BL']
newcol = 'oral_steroids_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]]).fillna(bigdf_nodup[cols[2]])
else:
    print(conflicts)
    print('conflicts!')


## Symptoms

Symptoms have matching column IDs for two of the three datasets.

In [146]:
# Problem swallowing
cols = ['PSWALL', 'pswall6mo_BL']
newcol = 'prob_swall_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]])
else:
    print('conflicts!')
    
# Food stuck
cols = ['FOODSTUCK', 'foodstuck6mo_BL']
newcol = 'food_stuck_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]])
else:
    print('conflicts!')

# Difficulty swallowing
cols = ['SWLLDIFF', 'diffswall6mo_BL']
newcol = 'diff_swall_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]])
else:
    print('conflicts!')

# Abdominal pain
cols = ['ABPAIN', 'abdpain6mo_BL']
newcol = 'abd_pain_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]])
else:
    print('conflicts!')

# Constipation
# Note: fundo patients do not have this metadata
newcols.append('const_all')
bigdf_nodup['const_all'] = bigdf_nodup['CONSTIP']

# Weight loss
cols = ['WTLOSS', 'wtloss6mo_BL']
newcol = 'wtloss_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]])
else:
    print('conflicts!')
    
# Food up
cols = ['FOODUP', 'foodup6mo_BL']
newcol = 'foodup_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]])
else:
    print('conflicts!')

# Chest pain
cols = ['CHSTPAIN', 'chstpain6mo_BL']
newcol = 'chest_pain_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]])
else:
    print('conflicts!')

# Asthma
cols = ['ASTH6', 'asthma6mo_BL']
newcol = 'asthma_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]])
else:
    print('conflicts!')

# Chronic cough
cols = ['COUGH6', 'COND48_BL']
newcol = 'chronic_cough_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]])
else:
    print('conflicts!')

# # Esophagitis
# cols = ['ABNLEGD', 'esophagitisYN']
# newcol = 'esophagitis_all'
# newcols.append(newcol)

# conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
# if conflicts == 0:
#     bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]])
# else:
#     print('conflicts!')

## History of infections

In [147]:
# Recent history of pneumonia
cols = ['PNEUMN', 'T0.PNEUM', 'pna6mo_BL']
newcol = 'pneum_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup['PNEUMN'].fillna(bigdf_nodup['T0.PNEUM']).fillna(bigdf_nodup['pna6mo_BL'])

# Ear infection
cols = ['EARINF', 'T0.EAR', 'OM6mo_BL']
newcol = 'ear_inf_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]])
else:
    print('conflicts!')

# Sinus infection
cols = ['SINUS', 'T0.SINUS']
newcol = 'sinus_inf_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]])
else:
    print('conflicts!')


## GI symptoms

For these, a value of -6 means "no data". We'll also replace -8 values, just in case they're in here too.

In [148]:
cols = ['PGSQsymp', 'PGSQSYMP_BL', 'PGSQSYMP']
newcol = 'pgsq_symp_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()

# Before merging, replace -6 with np.nan (so they don't get replaced to zero,
# like we're doing for all hte other columns)
bigdf_nodup[cols] = bigdf_nodup[cols].replace({'-6': np.nan, '-8': np.nan})
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]]).fillna(bigdf_nodup[cols[2]])
else:
    print('conflicts!')


In [149]:
cols = ['PGSQtotal', 'PGSQTOT_BL', 'PGSQTOT']
newcol = 'pgsq_total_all'
newcols.append(newcol)

conflicts = bigdf_nodup.apply(lambda row: identify_conflicts(row, cols), axis=1).sum()

# Before merging, replace -6 with np.nan (so they don't get replaced to zero,
# like we're doing for all hte other columns)
bigdf_nodup[cols] = bigdf_nodup[cols].replace({'-6': np.nan, '-8': np.nan})
if conflicts == 0:
    bigdf_nodup[newcol] = bigdf_nodup[cols[0]].fillna(bigdf_nodup[cols[1]]).fillna(bigdf_nodup[cols[2]])
else:
    print('conflicts!')


## Merge with existing metadata

In [171]:
fmeta = '../../data/clean/rosen.metadata.clean'
meta = pd.read_csv(fmeta, sep='\t', index_col=0)

In [172]:
meta.head()

,"If Yes, specify the symptom score","If yes, please indicate level",% time pH<4,% time pH<4:,A1. Subject ID number:,A2. Subject initials:,A3. What Cohort is the subject enrolled into?,A4. Aim(s) enrolled in?,A5. Date of initial/baseline visit/procedure (MM/DD/YYYY):,A5a. Date filled out(MM/DD/YYYY):,...,STUDYID,STUDY,AIM,SOURCE,PHMII,ACIDSUP,DATE,ppi_consolidated,mbs_consolidated,total_reads
01-112-7GI,NaN,NaN,NaN,NaN,01-112-7,DM,GERD,Aim 1,2014-04-14,2014-04-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271980
01-112-7RI,NaN,NaN,NaN,NaN,01-112-7,DM,GERD,Aim 1,2014-04-14,2014-04-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,244891
01-112-7TI,NaN,NaN,NaN,NaN,01-112-7,DM,GERD,Aim 1,2014-04-14,2014-04-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41988
01-164-7GI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89821
01-164-7RI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72671


In [173]:
meta.shape, meta['subject_id'].unique().shape

((455, 958), (220,))

In [174]:
meta.index.name = 'sample_id'
meta = meta.reset_index()

oldmetacols = ['sample_id', 'subject_id', 'site', 'mbs_consolidated']
newmetacols = newcols + ['metadata_id']
# Note: there are additional reflux metadata columns, but these are the
# ones with the most reliable measurements
refluxcols = ['Total number of reflux episodes (acid+non-acid)',
              'number of full colum events/total events',
              'percent proximal total',
              'percent distal total']

allmeta = pd.merge(bigdf_nodup[newmetacols + ['subject_id']], 
                   meta[oldmetacols + refluxcols].drop_duplicates(), 
                   left_on='subject_id', right_on='subject_id', 
                   how='right')

# Clean up the reflux cols
allmeta[refluxcols] = allmeta[refluxcols].replace('n/a', np.nan)
allmeta[refluxcols] = allmeta[refluxcols].astype(float)
# Some missing entries are wrongly entered as 1
allmeta['number of full colum events/total events'] = allmeta['number of full colum events/total events'].replace(1, np.nan)


print(allmeta.shape)

(455, 31)


In [175]:
allmeta['subject_id'].unique().shape

(220,)

In [176]:
print(newcols)

['age', 'gender_all', 'abx_all', 'ppi_all', 'h2blockers_all', 'inhaled_steroids_all', 'oral_steroids_all', 'prob_swall_all', 'food_stuck_all', 'diff_swall_all', 'abd_pain_all', 'const_all', 'wtloss_all', 'foodup_all', 'chest_pain_all', 'asthma_all', 'chronic_cough_all', 'pneum_all', 'ear_inf_all', 'sinus_inf_all', 'pgsq_symp_all', 'pgsq_total_all']


In [177]:
allmeta = allmeta[oldmetacols + newmetacols + refluxcols]
allmeta.head()

,sample_id,subject_id,site,mbs_consolidated,age,gender_all,abx_all,ppi_all,h2blockers_all,inhaled_steroids_all,...,pneum_all,ear_inf_all,sinus_inf_all,pgsq_symp_all,pgsq_total_all,metadata_id,Total number of reflux episodes (acid+non-acid),number of full colum events/total events,percent proximal total,percent distal total
0,04-006-9B,04-006-9,bal,Aspiration/Penetration,2.969863,F,0,1,0,0,...,,,,NaN,NaN,k23,NaN,NaN,NaN,NaN
1,04-009-2B,04-009-2,bal,Aspiration/Penetration,1.010959,M,0,0,0,0,...,1,0,0,1.12,0.61,k23,NaN,NaN,NaN,NaN
2,04-009-2G,04-009-2,gastric_fluid,Aspiration/Penetration,1.010959,M,0,0,0,0,...,1,0,0,1.12,0.61,k23,NaN,NaN,NaN,NaN
3,04-011-3B,04-011-3,bal,Normal,2.564384,F,1,1,0,0,...,1,0,0,0.29,0.26,k23,25.0,0.24,0.001231,0.004344
4,04-011-3G,04-011-3,gastric_fluid,Normal,2.564384,F,1,1,0,0,...,1,0,0,0.29,0.26,k23,25.0,0.24,0.001231,0.004344


# Fill NaN's

In [178]:
# Convert all values to strings and replace all missing values with NaN
allmeta = allmeta.replace({1: '1', 
                           0: '0', 
                           -9: '-9', 
                           11: '11'})

allmeta[newcols] = allmeta[newcols].replace({' ': np.nan, '-6': '0', '-9': np.nan, '-8': '0', '11': np.nan})

#allmeta[newcols]

In [179]:
allmeta.to_csv(outfile, sep='\t', index=False)

# Write subject-level version, consolidated with patients used in analyses

In [159]:
ptdf = pd.read_csv(ptfile, sep=',', index_col=0)
ptdf.head()

,bal,gastric_fluid,throat_swab,stool
04-080-7,True,True,True,False
01-299-7,False,False,True,False
04-167-8,False,True,False,False
04-087-1,True,True,True,False
042-6-F1,False,False,True,False


In [160]:
print(ptdf.shape, 
      allmeta.shape, 
      pd.merge(ptdf, 
               allmeta.drop(['sample_id', 'site'], axis=1).drop_duplicates(), 
               left_index=True, right_on='subject_id', how='left').shape
     )

((220, 4), (455, 27), (220, 29))


In [161]:
pt_meta =  pd.merge(
    ptdf, 
    allmeta.drop(['sample_id', 'site'], axis=1).drop_duplicates(), 
    left_index=True, right_on='subject_id', 
    how='left')
pt_meta.shape, pt_meta['subject_id'].unique().shape

((220, 29), (220,))

In [162]:
pt_meta = pt_meta[['subject_id'] + pt_meta.drop('subject_id', axis=1).columns.tolist()]
pt_meta.head()

,subject_id,bal,gastric_fluid,throat_swab,stool,mbs_consolidated,age,gender_all,abx_all,ppi_all,...,foodup_all,chest_pain_all,asthma_all,chronic_cough_all,pneum_all,ear_inf_all,sinus_inf_all,pgsq_symp_all,pgsq_total_all,metadata_id
67,04-080-7,True,True,True,False,NaN,7.413699,M,0,0,...,1,0,1,0,0,0,0,0.47,0.26,k23
425,01-299-7,False,False,True,False,NaN,11.301370,M,0,0,...,1,1,NaN,NaN,0,0,0,2.12,1.19,r01_baseline_v2
180,04-167-8,False,True,False,False,Aspiration/Penetration,5.079452,F,0,1,...,0,0,NaN,NaN,0,1,0,0.53,0.57,k23
80,04-087-1,True,True,True,False,NaN,2.775342,M,0,0,...,0,0,1,1,0,0,1,1.47,1.39,k23
444,042-6-F1,False,False,True,False,NaN,10.846575,M,0,1,...,1,1,0,0,0,NaN,NaN,2.088,2.73,fundo_v2


In [163]:
pt_meta.to_csv(pt_outmeta, index=False)

## For Rachel: list of patients who are aspirators

In [164]:
asp = 'Aspiration/Penetration'
print('\n'.join(pt_meta.query('mbs_consolidated == @asp')['subject_id'].tolist()))

04-167-8
04-157-4
04-201-8
03-124-8
02-298-5
04-255-1
04-274-9
04-294-8
04-064-5
04-182-6
04-077-1
02-184-5
04-172-0
04-139-5
04-154-9
04-036-5
04-287-1
04-291-3
04-200-1
04-054-3
04-309-9
04-257-3
04-156-8
04-006-9
04-072-1
03-121-5
04-138-4
02-143-2
04-278-5
04-112-4
04-276-7
04-299-7
04-240-2
04-166-7
03-094-5
04-303-7
02-186-0
03-089-3
04-133-5
04-247-3
04-009-2
13-117-4
04-262-5
04-259-2
12-160-1
02-183-7
03-102-4
